# Template: visualize spatially-resolved single-cell data with Vitessce

## Code to change

Places where you will need to edit the code are marked by `# TODO(template)` comments.

In [1]:
import os
import json
from os.path import join
from vitessce import (
    VitessceConfig,
    Component as cm,
    CoordinationType as ct,
    FileType as ft,
    AnnDataWrapper,
    OmeTiffWrapper,
    MultiImageWrapper,
    BASE_URL_PLACEHOLDER,
)
from vitessce.data_utils import (
    rgb_img_to_ome_tiff,
    multiplex_img_to_ome_tiff,
    optimize_adata,
    VAR_CHUNK_SIZE,
)
from anndata import read_h5ad
import pandas as pd
import numpy as np
from tifffile import imread
from skimage.draw import disk

## Variables to fill in

In [2]:
# TODO(template)
PATH_TO_INPUT_ANNDATA_H5AD = join('..', 'tutorials', 'spatial_single_cell', 'raw_data', 'human_lymph_node.h5ad')

# TODO(template): specify the file path for the input TIFF image
PATH_TO_INPUT_TIFF = join('..', 'tutorials', 'spatial_single_cell', 'raw_data', 'human_lymph_node.tiff')

# TODO(template)
PATH_TO_OUTPUT_ANNDATA_ZARR = join('.', 'processed_data', 'spatial_single_cell', 'output.anndata.zarr')

# TODO(template): specify some file paths for the converted OME-TIFF image and bitmask files
PATH_TO_OUTPUT_IMAGE_OME_TIFF = join('.', 'processed_data', 'spatial_single_cell', 'image.ome.tif')
PATH_TO_OUTPUT_IMAGE_PYRAMIDAL_OME_TIFF = join('.', 'processed_data', 'spatial_single_cell' 'image.pyramid.ome.tif')
PATH_TO_OUTPUT_BITMASK_OME_TIFF = join('.', 'processed_data', 'spatial_single_cell' 'bitmask.pyramid.ome.tif')

# TODO(template): specify a local folder to store the publication dataset.
PATH_TO_PUBLICATION_DATASET_DIRECTORY = join('.', 'my_publication_dataset')

# TODO(template): Provide a name for the publication dataset and this particular vignette.
PUBLICATION_DATASET_NAME = 'My nature paper'
PUBLICATION_DATASET_ID = 'my-nature-paper'
VIGNETTE_NAME = 'Spatial transcriptomics'
VIGNETTE_ID = 'vignette_visium'

# TODO(template): provide names and descriptions
CONFIG_NAME = 'My config'
CONFIG_DESCRIPTION = 'This dataset reveals...'
DATASET_NAME = 'My dataset'
IMG_NAME = 'My image'

## Validation

In [3]:
DATA_DIR = join(PATH_TO_PUBLICATION_DATASET_DIRECTORY, 'data', VIGNETTE_ID)
VIGNETTES_DIR = join(PATH_TO_PUBLICATION_DATASET_DIRECTORY, 'vignettes', VIGNETTE_ID)

if not os.path.isdir(DATA_DIR):
    os.makedirs(DATA_DIR, exist_ok=False)
if not os.path.isdir(VIGNETTES_DIR):
    os.makedirs(VIGNETTES_DIR, exist_ok=False)

## 1.1 Convert H5AD to AnnData-Zarr

In [4]:
adata = read_h5ad(PATH_TO_INPUT_ANNDATA_H5AD)

In [5]:
# TODO(template): Identify the scale factor required for aligning the image to the spatial coordinates.
scale_factor = 1 / 5.87
adata.obsm['spatial'] = (adata.obsm['spatial'] * scale_factor)

In [6]:
# Store the current observation index in a new column called "spot_id"
adata.obs = adata.obs.reset_index().rename(columns={"index": "spot_id"})
# Create an integer index starting at 1 (0 is reserved for the background)
adata.obs.index = list(range(1, adata.shape[0]+1))
adata.obs

,spot_id,in_tissue,array_row,array_col,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,total_counts_mt,log1p_total_counts_mt,pct_counts_mt,n_counts,clusters
1,AAACAAGTATCTCCCA-1,1,50,102,6732,8.814776,27944.0,10.237993,33.241483,41.071429,48.375322,57.572287,248.0,5.517453,0.887489,27944.0,1
2,AAACAATCTACTAGCA-1,1,3,43,6759,8.818778,25685.0,10.153702,23.507884,33.965349,42.631886,53.054312,239.0,5.480639,0.930504,25685.0,0
3,AAACAGAGCGACTCCT-1,1,14,94,7236,8.886962,31916.0,10.370894,21.948239,31.990224,41.101642,52.603710,348.0,5.855072,1.090362,31916.0,8
4,AAACAGCTTTCAGAAG-1,1,43,9,6890,8.837971,30932.0,10.339579,26.357817,38.132032,47.575327,57.590844,793.0,6.677083,2.563688,30932.0,2
5,AAACAGGGTCTATATT-1,1,47,13,7631,8.940105,31728.0,10.364986,22.462809,31.360313,39.696798,50.107161,287.0,5.662961,0.904564,31728.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3857,TTGTTTCACATCCAGG-1,1,58,42,5771,8.660774,19649.0,9.885833,27.131152,35.950939,44.429742,55.000254,209.0,5.347107,1.063667,19649.0,6
3858,TTGTTTCATTAGTCTA-1,1,60,30,6354,8.756997,21557.0,9.978502,24.238067,32.263302,40.265343,51.152758,208.0,5.342334,0.964884,21557.0,6
3859,TTGTTTCCATACAACT-1,1,45,27,6517,8.782323,22175.0,10.006766,21.506201,30.593010,39.098083,50.119504,192.0,5.262690,0.865840,22175.0,6
3860,TTGTTTGTATTACACG-1,1,73,41,4476,8.406708,12268.0,9.414831,24.078904,34.268014,43.193675,54.964134,121.0,4.804021,0.986306,12268.0,0


In [7]:
adata = optimize_adata(
    adata,
    # TODO(template): Specify the columns and keys that will be used in the visualization.
    obs_cols=["spot_id", "clusters"],
    var_cols=["highly_variable"],
    obsm_keys=["X_hvg", "spatial", "X_umap", "X_pca"],
    optimize_X=True,
    # Vitessce plays nicely with dense matrices saved with chunking
    # and this one is small enough that dense is not a huge overhead.
    to_dense_X=True,
)

/Users/mkeller/anaconda3/envs/vitessce-tutorial-env/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [8]:
adata.write_zarr(PATH_TO_OUTPUT_ANNDATA_ZARR, chunks=[adata.shape[0], VAR_CHUNK_SIZE])

## 1.2 Convert TIFF to OME-TIFF

In [9]:
img_arr = imread(PATH_TO_INPUT_TIFF)

# Update the array axes so they are in CYX order to enable conversion to OME-TIFF.
img_arr = img_arr.transpose((2, 0, 1))
img_arr.shape

(3, 2000, 1921)

In [10]:
rgb_img_to_ome_tiff(img_arr, PATH_TO_OUTPUT_IMAGE_OME_TIFF, axes="CYX", img_name="H & E Image")

In [11]:
!~/software/bftools/bfconvert -overwrite -tilex 128 -tiley 128 -pyramid-resolutions 2 -pyramid-scale 2 -compression LZW {PATH_TO_OUTPUT_IMAGE_OME_TIFF} {PATH_TO_OUTPUT_IMAGE_PYRAMIDAL_OME_TIFF}
# TODO(template): For larger images, you will want to comment out the above line and un-comment the line below,
# to increase the tile size (128 -> 512) and the number of pyramid resolutions (2 -> 6).
# !~/software/bftools/bfconvert -tilex 512 -tiley 512 -pyramid-resolutions 6 -pyramid-scale 2 -compression LZW {PATH_TO_OUTPUT_OME_TIFF} {PATH_TO_OUTPUT_PYRAMIDAL_OME_TIFF}

./processed_data/spatial_single_cell/image.ome.tif
OMETiffReader initializing ./processed_data/spatial_single_cell/image.ome.tif
Reading IFDs
Populating metadata
[OME-TIFF] -> ./processed_data/spatial_single_cellimage.pyramid.ome.tif [OME-TIFF]
Tile size = 128 x 128
	Converted 1/3 planes (33%)
	Converted 2/3 planes (66%)
	Converted 3/3 planes (100%)
Tile size = 128 x 128
	Converted 3/3 planes (100%)
[done]
7.977s elapsed (6.3333335+1184.8334ms per plane, 781ms overhead)


## 1.3 Generate OME-TIFF bitmask

In [12]:
bitmask_arr = np.zeros((img_arr.shape[2], img_arr.shape[1]), dtype=np.uint16)
bitmask_arr.shape

(1921, 2000)

In [13]:
radius = 10
for i in range(adata.shape[0]):
    x = adata.obsm['spatial'][i, 0]
    y = adata.obsm['spatial'][i, 1]
    bitmask_arr[disk((x, y), radius)] = i+1 # add one (0 is reserved for the background)

# Update the array axes so they are in CYX order to enable conversion to OME-TIFF.
bitmask_arr = bitmask_arr.transpose((1, 0)) # (y, x)
bitmask_arr = bitmask_arr[np.newaxis, :] # (c, y, x)
bitmask_arr.shape

(1, 2000, 1921)

In [14]:
multiplex_img_to_ome_tiff(bitmask_arr, ["Spots"], PATH_TO_OUTPUT_BITMASK_OME_TIFF, axes="CYX")

## 2. Configure the visualization

In [15]:
vc = VitessceConfig(schema_version="1.0.15", name=CONFIG_NAME, description=CONFIG_DESCRIPTION)

dataset = vc.add_dataset(name=DATASET_NAME).add_object(AnnDataWrapper(
    adata_path=PATH_TO_OUTPUT_ANNDATA_ZARR,
    # TODO(template): update the arrays of interest and where they are located in the AnnData object.
    obs_embedding_paths=["obsm/X_umap", "obsm/X_pca"],
    obs_embedding_names=["UMAP", "PCA"],
    obs_locations_path="obsm/spatial",
    obs_set_paths=["obs/clusters"],
    obs_set_names=["Leiden Cluster"],
    obs_feature_matrix_path="obsm/X_hvg",
    feature_filter_path="var/highly_variable",
)).add_object(MultiImageWrapper([
    OmeTiffWrapper(
        img_path=PATH_TO_OUTPUT_BITMASK_OME_TIFF,
        name="Spot segmentations",
        is_bitmask=True
    ),
    OmeTiffWrapper(
        img_path=PATH_TO_OUTPUT_IMAGE_PYRAMIDAL_OME_TIFF,
        name="H&E Image",
        is_bitmask=False
    ),
], use_physical_size_scaling=True))

# TODO(template): Update the views of interest.
spatial_colored_by_cluster = vc.add_view(cm.SPATIAL, dataset=dataset)
spatial_colored_by_expression = vc.add_view(cm.SPATIAL, dataset=dataset)

layer_controller = vc.add_view(cm.LAYER_CONTROLLER, dataset=dataset).set_props(disableChannelsIfRgbDetected=True)
spot_set_manager = vc.add_view(cm.OBS_SETS, dataset=dataset)
gene_list = vc.add_view(cm.FEATURE_LIST, dataset=dataset)
heatmap = vc.add_view(cm.HEATMAP, dataset=dataset).set_props(transpose=True)

# TODO(template): Update the layout of views.
vc.layout(
    (spatial_colored_by_cluster | spatial_colored_by_expression)
    / ((layer_controller | spot_set_manager) | (gene_list | heatmap))
);

In [16]:
# TODO(template): configure view coordinations and initial coordination values
spatial_views = [
    spatial_colored_by_cluster,
    spatial_colored_by_expression,
    layer_controller,
]
all_views = [
    *spatial_views,
    spot_set_manager,
    gene_list,
    heatmap
]

spatial_segmentation_layer_value = [{
    "type": "bitmask",
    "visible": True,
    "index": 0,
    "colormap": None,
    "transparentColor": None,
    "opacity": 1,
    "domainType": "Min/Max",
    "channels": [
        {
          "selection": { "c": 0, "t": 0, "z": 0 },
          "color": [255, 0, 0],
          "visible": True,
          "slider": [0, 1]
        }
    ]
}]
spatial_image_layer_value = [{
    "type": "raster",
    "index": 0,
    "colormap": None,
    "transparentColor": None,
    "opacity": 1,
    "domainType": "Min/Max",
    "channels": [
        {
          "selection": { "c": 0, "t": 0, "z": 0 },
          "color": [255, 0, 0],
          "visible": True,
          "slider": [0, 255]
        },
        {
          "selection": { "c": 1, "t": 0, "z": 0 },
          "color": [0, 255, 0],
          "visible": True,
          "slider": [0, 255]
        },
        {
          "selection": { "c": 2, "t": 0, "z": 0 },
          "color": [0, 0, 255],
          "visible": True,
          "slider": [0, 255]
        }
    ]
}]
vc.link_views(
    spatial_views,
    [ct.SPATIAL_IMAGE_LAYER, ct.SPATIAL_SEGMENTATION_LAYER],
    [spatial_image_layer_value, spatial_segmentation_layer_value]
)
vc.link_views(
    spatial_views,
    [ct.SPATIAL_ZOOM, ct.SPATIAL_TARGET_X, ct.SPATIAL_TARGET_Y],
    [-2.6, 1000, 1000]
)
vc.link_views(
    [spatial_colored_by_cluster, heatmap, spot_set_manager],
    [ct.OBS_COLOR_ENCODING],
    ["cellSetSelection"]
)
vc.link_views(
    [spatial_colored_by_expression, gene_list],
    [ct.OBS_COLOR_ENCODING, ct.FEATURE_SELECTION],
    ["cellSetSelection", ["CR2"]]
)

### Render the widget

In [17]:
vw = vc.widget()
vw

VitessceWidget(config={'version': '1.0.15', 'name': 'My config', 'description': 'This dataset reveals...', 'da…

## 3. Export the configuration and data

In [18]:
config_dict = vc.export(to="files", base_url=f'{BASE_URL_PLACEHOLDER}/data/{VIGNETTE_ID}', out_dir=DATA_DIR)

# Use `open` to create a new empty file at ./exported_data/vitessce.json
with open(join(VIGNETTES_DIR, "vitessce.json"), "w") as f:
    json.dump(config_dict, f)